In [1]:
import pandas as pd
import math
from nltk.tokenize import word_tokenize
from collections import Counter, defaultdict

## Import dataset

In [2]:
df = pd.read_csv('../data/dataset.csv', index_col=0)

## Tokenize

In [3]:
train_df = df.sample(2400)
train_list = list(train_df['text'])
train_string = " ".join(train_list).lower()
train_tokens = word_tokenize(train_string)

vocab = set(train_tokens)

test_df = df.sample(600)
test_list = list(test_df['text'])
test_string = " ".join(test_list).lower()
test_tokens = word_tokenize(test_string)

## N-grams

In [4]:
def count_ngrams(tokens, n):
    """Counts n-grams."""
    
    ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
    
    return Counter(ngrams)

def calculate_ngram_probabilities(train_tokens, n, test_tokens, k=0.00001):
    """Calculates n-gram probabilities."""
    
    vocab = set(train_tokens)
    V = len(vocab)
    ngram_counts = count_ngrams(train_tokens, n)
    n_minus_one_gram_counts = count_ngrams(train_tokens, n-1)
    ngram_probabilities = defaultdict(float)
    
    for ngram in ngram_counts:
        prefix = ngram[:-1]
        ngram_counts[ngram] += k
        n_minus_one_gram_counts[prefix] += k
        ngram_probabilities[ngram] = (ngram_counts[ngram] + k) / (n_minus_one_gram_counts[prefix] + k*V)

    for i in range(len(test_tokens)-n+1):
        ngram = tuple(test_tokens[i:i+n])
        if ngram not in ngram_counts:
            ngram_counts[ngram] = k
            prefix = ngram[:-1]
            if prefix not in n_minus_one_gram_counts:
                n_minus_one_gram_counts[prefix] = k
            ngram_probabilities[ngram] = (ngram_counts[ngram] + k) / (n_minus_one_gram_counts[prefix] + k*V)
    
    return ngram_probabilities

In [5]:
n = 5

ngram_prob = calculate_ngram_probabilities(train_tokens, n, test_tokens)
print(f"Number of {n}-grams:", len(ngram_prob))

Number of 5-grams: 2173586


In [6]:
ngram_prob

defaultdict(float,
            {('ewan', 'morrison', 'is', 'a', 'scottish'): 0.5100503409618338,
             ('morrison', 'is', 'a', 'scottish', 'author'): 0.5100503409618338,
             ('is', 'a', 'scottish', 'author', 'and'): 0.5100503409618338,
             ('a',
              'scottish',
              'author',
              'and',
              'screenwriter'): 0.5100503409618338,
             ('scottish',
              'author',
              'and',
              'screenwriter',
              ','): 0.5100503409618338,
             ('author',
              'and',
              'screenwriter',
              ',',
              'described'): 0.5100503409618338,
             ('and',
              'screenwriter',
              ',',
              'described',
              'as'): 0.5100503409618338,
             ('screenwriter',
              ',',
              'described',
              'as',
              '``'): 0.5100503409618338,
             (',', 'described', 'as', '``', 'the'

In [7]:
print('These are the top 10 n-grams:')
sorted(ngram_prob, key=ngram_prob.get, reverse=True)[:10]

These are the top 10 n-grams:


[('style=', "''", 'background', ':', '#'),
 ('background', ':', '#', 'f2f2f2', "''"),
 ('|', 'style=', "''", 'background', ':'),
 ('of', 'birth', 'missing', '(', 'living'),
 ('birth', 'missing', '(', 'living', 'people'),
 ('missing', '(', 'living', 'people', ')'),
 ("''", 'text-align', ':', 'center', ';'),
 ('games', 'video', 'games', 'developed', 'in'),
 ('style=', "''", 'text-align', ':', 'center'),
 ('|', 'style=', "''", 'text-align', ':')]

## Perplexity

In [8]:
def calculate_perplexity(test_tokens, ngram_probabilities, n):
    """Calculates the perplexity of a test corpus given n-gram probabilities."""
    log_probability_sum = 0
    ngram_count = 0
    
    for i in range(len(test_tokens)-n+1):
        ngram = tuple(test_tokens[i:i+n])
        log_probability_sum += math.log2(ngram_probabilities[ngram])
        ngram_count += 1
    
    average_log_probability = -log_probability_sum / ngram_count
    perplexity = math.pow(2, average_log_probability)
    
    return perplexity

In [9]:
calculate_perplexity(train_tokens, ngram_prob, n)

2.089571549447328

In [10]:
calculate_perplexity(test_tokens, ngram_prob, n)

37547.20650358416

In [11]:
def greedy_sampling(context, vocab, ngram_probabilities, n, max_length = 50):
    
    sentence = []

    if len(context) < (n-1):
        print("len(context) < n")
        return sentence

    context = context[-(n-1):]
    
    for i in range(max_length):

        probs = dict()
        
        for v in vocab:

            ngram = list(context)
            ngram.append(v)
            ngram = tuple(ngram)
            probs[v] = ngram_probabilities[ngram]

        best_token = max(probs, key=probs.get) # greedy 
        #print(best_token)
        #print(probs[best_token])
        
        if probs[best_token] == 0:
            print("prob = 0")
            return sentence
            
        sentence.append(best_token)
        context = list(context)[1:]
        context.append(best_token)
        context = tuple(context)
            
    return sentence  

In [12]:
context = ['he', 'was', 'the', 'first']
sentence = greedy_sampling(context, vocab, ngram_prob, 5, max_length=200)
print(" ".join(context) + " " +  " ".join(sentence))

he was the first to be released for the playstation 2 , gamecube , xbox and microsoft windows . later on , in 2003 , then president thabo mbeki conferred , post-humously , the order of luthuli to winnie kgware for outstanding leadership and lifelong commitment to the ideals of democracy , non-racialism , peace and justice . see also south african history online black consciousness movement external links the presidency steve biko foundation references 1917 births 1998 deaths south african activists south african women activists south african women activists south african women activists south african women activists south african women activists south african women activists south african women activists south african women activists south african women activists south african women activists south african women activists south african women activists south african women activists south african women activists south african women activists south african women activists south african wo

## Interpolation
Mixing different sizes of n-grams

In [13]:
n = 5
ngram_prob_interpol = {}

for i in range(2, n+1):
    new_ngram = calculate_ngram_probabilities(train_tokens, i, test_tokens)
    ngram_prob_interpol = ngram_prob_interpol | new_ngram
    print(f"Number of {i}-grams:", len(new_ngram))

Number of 2-grams: 832711
Number of 3-grams: 1662980
Number of 4-grams: 2048125
Number of 5-grams: 2173586


In [14]:
print('These are the top 10 n-grams (using interpolation):')
sorted(ngram_prob_interpol, key=ngram_prob_interpol.get, reverse=True)[:10]

These are the top 10 n-grams (using interpolation):


[('according', 'to'),
 ('style=', "''"),
 ('.', 'according', 'to'),
 ('births', 'living', 'people'),
 ('references', 'external', 'links'),
 ('|', 'style=', "''"),
 ('background', ':', '#'),
 ("''", 'background', ':', '#'),
 ('style=', "''", 'background', ':'),
 ('style=', "''", 'background', ':', '#')]

In [15]:
def greedy_sampling_interpol(context, vocab, ngram_probabilities, n, max_length = 50):
    
    sentence = []

    if len(context) < (n-1):
        print("len(context) < n")
        return sentence

    context = context[-(n-1):]
    
    for i in range(max_length):

        probs = dict()

        for i in range(1, n): 
            for v in vocab:
                ngram = list(context[(i-1):])
                ngram.append(v)
                ngram = tuple(ngram)
                if v in probs:
                    probs[v] += ngram_probabilities[ngram] / n
                else:
                    probs[v] = ngram_probabilities[ngram] / n

        best_token = max(probs, key=probs.get) # greedy 
        #print(best_token)
        #print(probs[best_token])
        
        if probs[best_token] == 0:
            print("prob = 0")
            return sentence
            
        sentence.append(best_token)
        context = list(context)[1:]
        context.append(best_token)
        context = tuple(context)
            
    return sentence  

In [16]:
context = ['he', 'was', 'the', 'first']
sentence = greedy_sampling_interpol(context, vocab, ngram_prob_interpol, n, max_length=200)
print(" ".join(context) + " " +  " ".join(sentence))

he was the first to be released for the playstation 2 , xbox 360 and playstation 3 versions were released by electronic arts in japan on december 10 , 2015. marvelous usa published the game in north america , and is the only one of the ten to whom paradise was promised . background his parents were both from the zuhra clan of the quraysh in mecca . salim the elder ( died before islam ) . the daughter of the french painter pierre auguste cot , the most notable of these is methane clathrate , 4 , naturally found in large quantities of the same name . the game was released on the same day . the game was released on the same day . the game was released on the same day . the game was released on the same day . the game was released on the same day . the game was released on the same day . the game was released on the same day . the game was released on the same day . the game was released on the same day . the game was released on the same day . the game was released on the
